In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab13.ipynb")

<img src="data6.png" style="width: 15%; float: right; padding: 1%; margin-right: 2%;"/>

# Lab 13 – APIs, Prompt Engineering

## Data 6

In [ ]:
from datascience import *
import numpy as np
from tqdm import tqdm
import json
from IPython.display import YouTubeVideo, HTML, display
from ipywidgets import interact, widgets
%matplotlib inline

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Part 1: Cohen's Kappa

**Cohen’s Kappa** is a measure of agreement between two annotators (or raters) who independently classify items into categories.

Unlike simple agreement rates (e.g., how often the two raters agree), **Cohen’s Kappa adjusts for chance agreement,** that is, how often two people might agree just by random guessing.

Read the [Data 6 Notes](https://data6.org/notes/20-coding/cohens-kappa.html) for a full description of this measure, along with a walkthrough example.

## Cohen’s Kappa Formula

Cohen's kappa measures the agreement between two raters who each classify items into a set of mutually exclusive categories. Here is the mathematical notation of Cohen's Kappa:

$$\kappa = \frac{p_o - p_e}{1 - p_e}$$

Where:
- $p_o$ = observed agreement rate, i.e., how often the raters agreed
- $p_e$ = random agreement rate, i.e., how likely the raters would agree just by randomly guessing.

We will not go into the probability in this course, but here's the idea. Cohen's Kappa is a **ratio** of two values:

$$\kappa = \dfrac{\text{observed agreement rate} - \text{random agreement rate}}{1 - \text{random agreement rate}}$$

If the raters are in complete agreement, then the observed agreement rate $p_o = 1$ and $\kappa = 1$. If the raters only agree randomly, then the observed agreement rate $p_o = p_e$ and $\kappa = 0$. It is possible for $\kappa < 0$, which can occur if there is really no relationship between the raters' rankings, or if raters are biased in their ratings.


**In general, we will **not** ask you to manually compute Cohen's Kappa**. We will see that there is a convenient Python library called sklearn for computing Cohen's Kappa in practice. However, it is good to first internalize this idea of "random chance" (as we will see in the first question) with a manual computation before we rely on the library (as we will see in the second question).

<hr style="border: 1px solid #fdb515;" />

# Question 1: Binary Classification


In this question, we will manually compute Cohen's Kappa on a binary labeling task. This question just uses very simple Python (like a calculator!), but make sure you understand how we compute each part of the Cohen's Kappa formula. This example is sourced from the [Wikipedia article on Cohen's Kappa](https://en.wikipedia.org/wiki/Cohen%27s_kappa).

Suppose that you were analyzing data related to a group of 50 people applying for a grant. Each grant proposal was read by two readers and each reader decides either "Yes" or "No" to the proposal. Suppose the summary of readers A and B's decisions were as follows:

|             | B: Yes | B: No |
| ---         | --- | --- |
| <b>A: Yes</b> | 20 | 5 | 
| <b>A: No</b>  | 10 | 15 |


This means:
- Both A and B agreed on 35 grants:
    - Both said **Yes** on 20 grants.
    - Both said **No** on 15 grants.
- A and B disagreed on 15 grants:
    - A said **Yes**, B said **No** on 5 grants.
    - A said **No**, B said **Yes** on 10 grants.

Run the below cell to load the `grant_decisions` table with A and B's decision for each grant, one per row.

In [ ]:
# just run this cell
grant_decisions = Table.read_table("grants.csv")
grant_decisions.show(5)

We can verify that we can recreate the "matrix" above as a pivot table of `grant_decisions` with differently labeled columns and reordered rows:

In [ ]:
# just run this cell
# verify the values make sense to you based on the matrix in the description above
grant_decisions.pivot("B", "A")

---

## Question 1a: Compute observed agreement rate, $p_o$

The rate of observed agreement $p_o$ is the fraction of grants for which A and B actually agreed on their decision, i.e., they both decided "yes" or both decided "no". Use the `grant_decisions` table to compute this value and assign it to `p_o`.

_Hints_:
* We have used `num_yes_yes` and `num_no_no` to get the count of rows in `grant_decisions` for which A and B both decided "yes" and both decided "no", respectively. Our expression used method chaining. We then used these values to compute `p_o`, factoring in the total number of grants.
* If you are still stuck, refer to the [Wikipedia article](https://en.wikipedia.org/wiki/Cohen%27s_kappa#Simple_example) which walks through the computation for this question.

In [ ]:
num_yes_yes = ...
num_no_no = ...
p_o = ...
p_o

In [ ]:
grader.check("q1a")

---

## Question 1b: Compute random agreement rate, $p_e$

The rate of random agreement $p_e$ is the _hypothetical_ (i.e., expected) fraction of grants for which A and B might randomly agree. That is, if A had randomly voted yes on agreements _based on A's observed "yes" rate_, and B had randomly also voted yes on agreements _based on B's observed "yes" rate_, then sometimes A and B might agree in their decisions purely based on chance.

Read the [Data 6 Notes](https://data6.org/notes/20-coding/cohens-kappa.html) for a suggested approach to computing $p_e$, rooted in probability theory (out of scope). Use the `grant_decisions` table to compute the random agreement rate and assign it to `p_e`.

_Hints_:
* We have defined `rate_yes_a` and `rate_yes_b` to get the "yes" rates of A and B (Step 1).
* We then used these rates to compute `chance_yes_yes` and `chance_no_no` to compute probabilities in Steps 2 and 3.
* Finally, we used these chance probabilities to compute the random agreement rate in Step 4.
* If you are still stuck, refer to the [Wikipedia article](https://en.wikipedia.org/wiki/Cohen%27s_kappa#Simple_example) which walks through the computation for this question.

In [ ]:
rate_yes_a = ...
rate_yes_b = ...
chance_yes_yes = ...
chance_no_no = ...
p_e = ...
p_e

In [ ]:
grader.check("q1b")

---

## Question 1c: Compute Cohen's Kappa, $\kappa$

Use your values of $p_o$ and $p_e$ to compute Cohen's Kappa and assign it to `kappa_manual`. Here is the formula again:


$$\kappa = \frac{p_o - p_e}{1 - p_e}$$

Again, we have included the text description of this formula in case it is easier to work through:

$$\kappa = \dfrac{\text{observed agreement rate} - \text{random agreement rate}}{1 - \text{random agreement rate}}$$

As per the [Wikipedia article](https://en.wikipedia.org/wiki/Cohen%27s_kappa#Simple_example), this value should be $0.4$.

In [ ]:
kappa_manual = ...
kappa_manual

In [ ]:
grader.check("q1c")

---

## Question 1d: Determine Agreement Level

Finally, use the Landis and Koch (1981, [DOI](https://doi.org/10.2307/2529310)) table to determine the level of agreement.

| $\kappa$ | Agreement |
| --- | --- |
| $ < 0$ | no agreement |
| $ 0-0.2$ | poor |
| $ 0.21-0.4 $ | fair | 
| $ 0.41-0.6 $ | moderate |
| $ 0.61 - 0.8 $ | good |
| $ 0.8 - 1.0 $ | near-perfect agreement |

Assign `agreement` to one of the Agreement strings above based on your value of `kappa_manual`.

In [ ]:
agreement = ...

In [ ]:
grader.check("q1d")

As mentioned earlier, in practice we do not manually compute Cohen's Kappa. As you may have gathered from your work so far, manual computation is fraught with many potential missteps, because there are many intermediate rates and probabilities to compute.

Instead, many instructors use the scikit-learn library, or **sklearn**, a Python package that is a toolkit for many scientific and machine learning applications. We will see that there is a much shorter way of reliably computing Cohen's Kappa. Onwards!

<hr style="border: 1px solid #fdb515;" />

# Question 2: scikit-learn (sklearn)

The Cohen's Kappa metric is provided in the `cohen_kappa_score` function.
    
Run the below cell to import the `cohen_kappa_score` from the `sklearn` library.

In [ ]:
# just run this cell
from sklearn.metrics import cohen_kappa_score

---

## Question 2a

Read about how to use this function in the [sklearn docs](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html). **Pay special attention to the argument types of `y1` and `y2`** (they are arrays).

Then, in the cell below, provide the appropriate columns of `grant_decisions` to pass in as arguments to the `cohen_kappa_score` function call. After running your cell, `kappa_sk` should be the same (or close to the same) value as the `kappa_manual` that you computed earlier.

In [ ]:
kappa_sk = cohen_kappa_score(..., ...)
kappa_sk

In [ ]:
grader.check("q2a")

---

## Question 2b

The `sklearn` function is flexible and can handle cases where the classification or labeling task involves more than just binary labels.

Suppose we had a different group of 50 people applying for a grant, where each grant proposal was read by two readers. However, these two readers C and D can decide either "Yes", "No", or "Abstain" for each proposal. Note that now there are **three** categories!

| | D: yes	| D: no	| D: abstain |
| --- | --- | --- | --- |
| <b>C: yes</b> |	11 |	12 |	1 |
| <b>C: no</b> |	15 |	5 |	4 |
| <b>C: abstain</b> |	0 |	2 |	0 |

Run the two cells below to load in the `grant_decisions_multi` table and reconstruct the matrix above, again with rows/columns potentially shuffled and relabeled.

In [ ]:
# just run this cell
grant_decisions_multi = Table.read_table("grants_multi.csv")
grant_decisions_multi.show(5)

In [ ]:
# just run this cell
# verify the values make sense to you based on the matrix in the description above
grant_decisions_multi.pivot("D", "C")

It would be challenging to compute Cohen's Kappa in this case by hand. Instead, the `cohen_kappa_score` function from the sklearn library makes it easy.

In the cell below, provide the appropriate columns of `grant_decisions_multi` to pass in as arguments to the `cohen_kappa_score` function call, and assign the Cohen's Kappa score to `kappa_multi`. Your approach should be structurally *very similar* to what you did in the previous part.

In [ ]:
kappa_multi = cohen_kappa_score(..., ...)
kappa_multi

In [ ]:
grader.check("q2b")

Note that in this case, we get a **negative** Cohen's Kappa score. This means that readers C and D reached very little agreement. We can see this in the matrix above, where there are more grants for which C and D disagree (one votes "yes", one votes "no") compared to those for which they agree (both vote "yes" or both vote "no").

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Part 2: The Google Gemini API

This part gives you your first experience using the Gemini Python API. We walk through the official tutorials and try to abstract away the complicated parts of the official documentation.

Resources:

* [Gemini API Quickstart](https://ai.google.dev/gemini-api/docs/quickstart)
* [Google Gen AI Python SDK](https://googleapis.github.io/python-genai/)
* [Gemini API: Prompting Strategies](https://ai.google.dev/gemini-api/docs/prompting-strategies)
* [Ziem et al., Table 1](https://direct.mit.edu/view-large/figure/4722326/coli_a_00502_i004.tif): LLM Prompting Guidelines to generate consistent, machine-readable outputs for CSS tasks. (very useful for project!)


## [Tutorial] Install Google/Gemini Python API

Google's Gemini API uses a Python interface. This is unlike the Genius and Wikipedia APIs, which used special URLs to access data.

To access the Gemini API, we install the `google-genai` and related `google` packages. Run the cell below to install.

In [ ]:
# just run this cell
!pip install google-genai


---

# Question 3: Gemini API Key

We shared your API key with you through email. This API key allows Gemini to recognize who is using their API. In `api_key.py`, set `my_client_access_token` to be the string of alphanumeric characters that we provided you. 

<div class="alert alert-block alert-warning">

_**Important Note**_:

 Please DO NOT share your API key outside of this class. We will disable your API key (1) if you misuse it and/or exceed the free usage tier during the project, and (2) for all students after the semester has ended. If you'd like to play around with your code after the term, you'll have to get your own API key.** Ask us how to do this!

</div>

Follow the corresponding instructions in the [Data 6 Notes](https://data6.org/notes/18-html/genius.html) to navigate to the `api_key.py` file and edit it.

After you have updated `api_key.py` (the file within this assignment directory), running the below line should load your API token into `GOOGLE_API_KEY`.

In [ ]:
# before running this cell, make sure that you have updated api_key.py with your API key
%reload_ext autoreload
%autoreload 2
import api_key
GOOGLE_API_KEY = api_key.my_client_access_token

In [ ]:
grader.check("q3")

<hr style="border: 1px solid #fdb515;" />

# [Tutorial] Our first (two) API request(s)

The APIs that we've worked with so far (Genius, Wikipedia) have been what is called [RESTful APIs](https://en.wikipedia.org/wiki/REST), where we specify a URL to access structured data. On the other hand, we are going to directly use a Python library API, developed by Google as part of their Gemini Software Development Kit (Gemini SDK).

Using a Python API means that we will make a Python client object using Google's `genai` package, then call that object's methods to get data from Gemini. The Python API approach is often more convenient than the RESTful API approach when (1) the input and output are both quite flexible in format, as is the case for Generative AI data, and (2) when we need to make a lot of different calls to the API, perhaps within different functions.

Run the cell below to import Google's `genai` Python module:

In [ ]:
# just run this cell
from google import genai

We will look at the example API request listed in the [Gemini Quickstart Documentation](https://ai.google.dev/gemini-api/docs/quickstart) ("Make your first request"), which you are welcome (and encouraged!) to browse.

### [Tutorial] From chat interface to API

Consider the chat prompt shown in the screenshot, as well as (the start of) the model's response.

<img src="prompt-chat.png" alt="A screenshot of a Google Gemini chat prompt. Prompt is 'Explain how AI works in a few words.'. Response from Gemini chat is long but gets at the idea." width="800">

An **AI chatbot** (like what is shown in the screenshot) is an application *on top of* a **large language model (LLM)**. The **LLM** is what takes in user prompts and returns text responses. The **chatbot** is what filters input, perhaps converting and loading files with additional prompts, and returns filtered LLM responses back, perhaps with some HTML or Markdown formatting.

Let's break down what is happening in the above screenshot:

* The user **prompt**: "Explain how AI works in a few words."
* The model **response**: "AI works by using algorithms to analyze..."
* The specified **model**: Here, it is "Fast" (note the dropdown in the bottom right). The other option is "Thinking." We'll discuss this more later.

### [Tutorial] Create the client then make the request

Run the next cell, which uses Gemini's Python API to directly query the Gemini LLM with this prompt. We describe the behavior below the cell, so just run the cell first.

In [ ]:
# just run this cell

# first, create the client
client = genai.Client(api_key=GOOGLE_API_KEY)

# then make the API request
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words"
)
print(response.text)

What just happened (and how you would make these API calls):

1. Make an **API client**: Make a Gemini client object called `client`, which passes in your API key. (We will do this for you usually.) This client lets you access different API calls via its methods. We will focus on one specific method in this class, `generate_content`.
1. Make the **API request**. Call `generate_content`, available in via `client.models`. This cell takes in several arguments, which we specify by name:

    * `model`: The underlying large language model. Here we specify Gemini 2.5 Flash, which is equivalent to the "Fast" option in the Gemini chatbot.
    * `contents`: The **prompt**: "Explain how AI works in a few words." We discuss the type of `contents` later.
1. Receive a **response**. `response` is a [Gemini-specific object type](https://ai.google.dev/gemini-api/docs/quickstart). The details are too complicated for this course. Instead, we focus on the value **`response.text`**, the LLM response string itself.

### [Tutorial] Make another API request

Once we have created the API client, we do not need to remake the client. Instead, we can just make more calls to `generate_content`.

Run the example cell below to run the same prompt. Remember—**LLMs are random response generators**, so you will likely get a different response back!

In [ ]:
# just run this cell

# client is already created, so just call generate_content
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words"
)
print(response.text)

<hr style="border: 1px solid #fdb515;" />

# Question 4 – Understand the API Request

Let us examine the structure of this API request, which is effectively just a fancy function call.

---

## Question 4a

In the `client.models.generate_content` function call above, what is the data type of the argument passed to `model`? Assign `q4a` to an integer corresponding to the appropriate choice below.

1. Integer
2. Response
3. String
4. Array
5. List

In [ ]:
q4a = ...

In [ ]:
grader.check("q4a")

---

## Question 4b

Consider the response to the API request. In the example code, what is the data type of `response.text`? Assign `q4b` to an integer corresponding to the appropriate choice below.

1. Integer
2. Response
3. String
4. Array
5. List

In [ ]:
q4b = ...

In [ ]:
grader.check("q4b")

---

## Question 4c

In the `client.models.generate_content` API call above, what is the data type of the argument passed to `contents`? Assign `q4c` to an integer corresponding to the appropriate choice below.

1. Integer
2. Response
3. String
4. Array
5. List

In [ ]:
q4c = ...

In [ ]:
grader.check("q4c")

<hr style="border: 1px solid #fdb515;" />

# Question 5 – Your Turn

In the cell below, make an API request to see what the best restaurant in Berkeley is.

Make this request by calling `client.models.generate_content`. We recommend copying the example prompt above, then modifying `contents` to fit your needs. Finally, assign `restaurant_str` to the response string (done for you).

In [ ]:
response = ...

restaurant_str = response.text # we have done this for you
print(restaurant_str)

In [ ]:
grader.check("q5")

<hr style="border: 1px solid #fdb515;" />

# Question 6 – Different Gemini Models

You can specify different Gemini models to get different responses. Some more advanced models will produce higher-quality responses, though with the tradeoff that these responses take longer to get.

We recommend two models for this course, which are relatively fast and inexpensive:
 * "Fast" **Gemini 2.5 Flash** model (`"gemini-2.5-flash"`). Pretty fast, though depending on the time of day responses might still take a few seconds.
 * "Thinking" **Gemini 2.5 Pro** model (`"gemini-2.5-pro"`). More in-depth responses, though responses will take significantly longer than Fast responses.

In the cell below, paste your prompt from earlier and modify the model to use Gemini 2.5 Pro. Assign `restaurant_str_pro` to the response string (done for you). Note that it may take up to a minute or two to get a response back.

In [ ]:
response = client.models.generate_content(
    model= ...
    contents= ...
) 

restaurant_str_pro = response.text # we have done this for you
print(restaurant_str_pro)

In [ ]:
grader.check("q6")

**Discuss** (no response needed): Compare the responses from Gemini 2.5 Flash and Gemini 2.5 Pro. Are there perceivable differences in length? quality? tone? format? credibility?


<div class="alert alert-block alert-warning">

_**Important Note**_:

Gemini Pro not only has longer request times; it is also more expensive per request. We **strongly** recommended testing out prompts with **Gemini 2.5 Flash**. Once you are satisfied with your prompt, only then should you change models. (Otherwise you'll be stuck waiting for a while in-between requests...)

</div>

We use **Gemini 2.5 Flash** for the remainder of this notebook.

<hr style="border: 1px solid #fdb515;" />

# [Tutorial] Providing more context for prompts

From [Wikipedia](https://en.wikipedia.org/wiki/Prompt_engineering): **Prompt engineering** is the process of structuring or crafting a **prompt** (natural language text instruction) in order to produce better outputs from a generative artificial intelligence (AI) model.

Depending on how you wrote your prompts above, you may find that the LLM responses were too general. For example, the top recommendations might not be ideal for college students (Chez Panisse is pretty pricey for a weeknight dinner...). A common method in prompt engineering a better response is to provide more **context**. Context can help define:

* What information you are looking for in the response ("Only include cheap meals under $15, and only consider restaurants that are open past 9pm.")
* How long you want the response to be ("Limit your response to 200 words.")
* What tone you want the response to have ("Imagine you are a UC Berkeley student talking to a fellow classmate.")
* How to structure the response: markdown, comma-separated, JSON, etc. ("Format your response as a bulleted list.")
* Include linebreaks (with newline characters `'\n'`, or use multi-line strings) to delineate different aspects of the prompt

One way to provide detailed context is to just pass in a very, very long string as your prompt.

Run the cell below for an example. Note that the triple quotes (`"""`) allows you to specify a multi-line string, with line breaks.


In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""Imagine you are a UC Berkeley student talking to a fellow classmate.
    
    What is the best restaurant in Berkeley? Only include cheap meals under $15,
    and only consider restaurants that are open past 9pm.

    Format your response as a bulleted list. Limit your response to 200 words.
    """
)

print(response.text)

<br/><br/>

As we've seen in this class time and time again, we want to **reuse** pieces of code, data, etc. The same is true of our contexts! One formatting context may be very useful for an entirely different application, and it would be great to use those same strings.

The Gemini API supports multiple argument types for `content`. For this course, we focus on providing additional **context** to our prompts by passing in a **list of strings** to `content`. This allows us to reuse context parts. It is also comparable to separating context parts with linebreaks.

Run the cell below, which specifies the same prompt as before but now with a list of strings passed into `content`. (Again, because LLMs are random text generators, the response may be different from before.)

In [ ]:
# just run this cell
context_character = "Imagine you are a UC Berkeley student talking to a fellow classmate."
context_format = "Format your response as a bulleted list. Limit your response to 200 words."

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=[
        context_character,
        """
        What is the best restaurant in Berkeley? Only include cheap meals under $15,
        and only consider restaurants that are open past 9pm.    
        """,
        context_format
    ]
)

print(response.text)

## Prompt Engineering Resources

There are many guides out there to good prompt engineering. Here are a few we recommend:

* [Gemini API: Prompting Strategies](https://ai.google.dev/gemini-api/docs/prompting-strategies)
* [Ziem et al., Table 1](https://direct.mit.edu/view-large/figure/4722326/coli_a_00502_i004.tif): LLM Prompting Guidelines to generate consistent, machine-readable outputs for CSS tasks. (very useful for project!)

<!-- BEGIN QUESTION -->

<hr style="border: 1px solid #fdb515;" />

# Question 7 – Your Turn

Now, it's your turn. Let's review functions and conditionals together. Consider the [Data 6 Fall 2025 Quiz 3](https://data6.org/fa25/exams/quizzes/fa25-quiz3.pdf) `hot_or_not` question on page 4. We'd like to write one new questions identical in structure to Q2.1-2.2 but with a different argument to the function call.

Using the Prompt Engineering Resources above, write an API request that returns these two new questions. Try to prevent the LLM from giving you the answer—it should just give the same four multiple choice options!

Your updated prompt should also do the following:
1. Include a brief context (15 words or less) 
2. Enumerate options as alphabetical multiple choice, separated by a new line.
3. Specify constraints
4. Specify actions under uncertainty
5. Provide at least 1 example of expected input and output

You should use Gemini 2.5 Flash, not Pro. Assign `question_str` to the response string (done for you).

In [ ]:
context_hot_or_not = """def hot_or_not(rank):
    if rank < 50:
        return "hot"
    elif rank > 50:
        return "not bad"
""" # provided for you
context_example = ... 
response = ...
question_str = response.text # we have done this for you
print(question_str)

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Part 3: Final Project Planning

<hr style="border: 1px solid #fdb515;" />

# Question 8

Refer to [the announcement in Lecture 20](https://docs.google.com/presentation/d/1-ljp0mvgDFZYQgh5gkuSMl6oYSKxHZUwiB-nGoG4EPs/edit?slide=id.g3a676c0168b_6_11#slide=id.g3a676c0168b_6_11), which outlines the final project.

The Final Project is partnered. **Fill out the below form as part of lab credit this week.**

> [Data 6 Fall 2025 Final Project Partner Matching Form](https://docs.google.com/forms/d/e/1FAIpQLSdyhmG1bms5yq9rY8dAi4RYhybJua9esN8NoZ76ugOan3f1Uw/viewform?usp=header)

Guidelines:

* If you have a project partner in mind (even if from a different section), both partners must fill out the form and specify each other as partner.
* If you do not have a partner in mind, fill out the form to indicate your project partner preferences and we will match you, prioritizing similar students who are in the same section.
* If you do not fill out the form, we will randomly match you, prioritizing students who are in the same section.

There is nothing else to do for this question beyond filling out the Google Form.

## Pets of Data 6

Congratulations on completing Lab 13!

<img src="portobello_caramello.png" width="50%">

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)